# Segmentation in multiple channels: large mixed-channel object + red puncta

This notebook:
- loads the example image from GitHub (falls back to a synthetic image if download fails)
- splits the RGB channels and computes a mixed-channel image (mean of all channels)
- segments the large structure from the mixed-channel image (not just green)
- segments small structures from the red channel
- fills holes in both segmentations
- creates outlines and overlays them on the original image
- saves all intermediate results and figures to disk

Requested colors for outlines:
- red-channel blobs: green outlines
- mixed-channel large object: yellow outline

In [1]:
# Imports and small helpers
import os
import numpy as np
from skimage.io import imread, imsave
from skimage import color, filters, morphology, measure, segmentation, util, exposure
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

try:
    import napari_simpleitk_image_processing as nsitk
except Exception:
    nsitk = None
try:
    import napari_segment_blobs_and_things_with_membranes as nsbatwm
except Exception:
    nsbatwm = None

os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)

def to_uint8(img):
    img = np.asarray(img)
    if img.dtype == np.uint8:
        return img
    if img.dtype == np.bool_:
        return (img.astype(np.uint8)) * 255
    if img.dtype in (np.float32, np.float64):
        img = exposure.rescale_intensity(img, out_range=(0, 1))
        return (img * 255).astype(np.uint8)
    # generic fallback
    img = exposure.rescale_intensity(img, out_range=(0, 255)).astype(np.uint8)
    return img

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_tools_menu/__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")
/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/napari_skimage_regionprops/_table.py:7: UserWarning: No Qt bindings could be found
  warnings.warn(str(e))


## Load the image (from GitHub) or synthesize a fallback
If downloading fails (e.g. offline), we generate a small synthetic two-channel-like image to demonstrate the workflow reproducibly.

In [2]:
url = 'https://github.com/user-attachments/assets/a1501925-24ca-48ed-a222-b5d8a4bb0ea4'
try:
    image_rgb = imread(url)
    if image_rgb.ndim == 2:
        image_rgb = color.gray2rgb(image_rgb)
    if image_rgb.shape[-1] == 4:
        # convert RGBA to RGB
        image_rgb = to_uint8(color.rgba2rgb(image_rgb))
    else:
        image_rgb = to_uint8(image_rgb)
    source = 'downloaded'
except Exception:
    # Create a synthetic example resembling a long process with red puncta
    from skimage.draw import line as skline, disk
    h, w = 512, 768
    base = np.zeros((h, w), dtype=np.float32)
    rr, cc = skline(20, w//2-10, h-30, w//2+15)
    base[rr, cc] = 1.0
    base = ndi.gaussian_filter(base, 3.0)
    base = exposure.rescale_intensity(base, out_range=(0, 1))
    red = np.zeros_like(base)
    rng = np.random.default_rng(0)
    ys = np.linspace(60, h-60, 28)
    for y in ys:
        x = w//2 + rng.integers(-10, 10)
        r = int(rng.integers(2, 4))
        rrs, ccs = disk((int(y)+rng.integers(-2, 2), int(x)), r, shape=base.shape)
        red[rrs, ccs] = 1.0
    red = ndi.gaussian_filter(red, 1.0)
    rgb = np.dstack([np.clip(base + red, 0, 1), base, base])
    image_rgb = to_uint8(rgb)
    source = 'synthetic'

imsave('data/issue_314_input.png', image_rgb)

## Split channels and compute a mixed-channel image
- R, G, B channels are saved separately.
- The large structure will be segmented from the mixed image (mean of R, G, B), as requested by @haesleinhuepf to use the mix instead of green.

In [3]:
r = image_rgb[..., 0]
g = image_rgb[..., 1]
b = image_rgb[..., 2]
mix = to_uint8(np.mean(image_rgb.astype(np.float32)/255.0, axis=2))

imsave('results/channel_R.png', r)
imsave('results/channel_G.png', g)
imsave('results/channel_B.png', b)
imsave('results/mix_all_channels.png', mix)

# Optional overview figure saved to disk
fig, axs = plt.subplots(1, 4, figsize=(12, 3))
axs[0].imshow(image_rgb); axs[0].set_title('Original'); axs[0].axis('off')
axs[1].imshow(r, cmap='Reds'); axs[1].set_title('R'); axs[1].axis('off')
axs[2].imshow(g, cmap='Greens'); axs[2].set_title('G'); axs[2].axis('off')
axs[3].imshow(mix, cmap='gray'); axs[3].set_title('Mix (mean RGB)'); axs[3].axis('off')
fig.tight_layout()
fig.savefig('results/overview_channels.png', dpi=200)
plt.close(fig)

## a) Segment the big structure from the mixed-channel image
Steps: denoise → Otsu threshold → fill holes → keep largest component. Save binary mask as image and NumPy array.

In [4]:
# Denoise (Gaussian blur)
if nsbatwm is not None:
    den_mix = nsbatwm.gaussian_blur(mix, sigma=1.0)
else:
    den_mix = to_uint8(filters.gaussian(mix, sigma=1.0))

# Threshold (Otsu)
if nsitk is not None:
    bin_large = nsitk.threshold_otsu(den_mix) > 0
else:
    t = filters.threshold_otsu(den_mix)
    bin_large = den_mix > t

# Fill holes and cleanup
bin_large = ndi.binary_fill_holes(bin_large)
bin_large = morphology.remove_small_objects(bin_large, min_size=1000)

# Keep largest component only (the big structure)
labels = measure.label(bin_large)
if labels.max() > 0:
    sizes = np.bincount(labels.ravel())
    sizes[0] = 0
    keep = sizes.argmax()
    bin_large = labels == keep

imsave('results/mask_large_mixed.png', to_uint8(bin_large))
np.save('results/mask_large_mixed.npy', bin_large)

# Quick diagnostic figure
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
axs[0].imshow(mix, cmap='gray'); axs[0].set_title('Mix'); axs[0].axis('off')
axs[1].imshow(den_mix, cmap='gray'); axs[1].set_title('Denoised'); axs[1].axis('off')
axs[2].imshow(bin_large, cmap='gray'); axs[2].set_title('Large mask'); axs[2].axis('off')
fig.tight_layout()
fig.savefig('results/segment_large_mixed_steps.png', dpi=200)
plt.close(fig)

## b) Segment small structures from the red channel
Steps: white top-hat background removal → Otsu threshold → remove small objects → fill holes. Save binary mask as image and NumPy array.

In [5]:
# Background removal on red channel
r_float = r.astype(np.float32) / 255.0
if nsbatwm is not None:
    r_filt = nsbatwm.white_tophat((r_float * 255).astype(np.uint8), radius=7).astype(np.float32) / 255.0
else:
    selem = morphology.disk(7)
    r_filt = morphology.white_tophat(r_float, selem)
r_filt = exposure.rescale_intensity(r_filt, out_range=(0, 1))

# Threshold (Otsu)
if nsitk is not None:
    bin_red = nsitk.threshold_otsu(to_uint8(r_filt)) > 0
else:
    t_red = filters.threshold_otsu(r_filt)
    bin_red = r_filt > t_red

# Cleanup: keep puncta, fill holes
bin_red = morphology.remove_small_objects(bin_red, min_size=20)
bin_red = ndi.binary_fill_holes(bin_red)

imsave('results/mask_red_puncta.png', to_uint8(bin_red))
np.save('results/mask_red_puncta.npy', bin_red)

# Quick diagnostic figure
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
axs[0].imshow(r, cmap='Reds'); axs[0].set_title('Red'); axs[0].axis('off')
axs[1].imshow(r_filt, cmap='gray'); axs[1].set_title('Top-hat filtered'); axs[1].axis('off')
axs[2].imshow(bin_red, cmap='gray'); axs[2].set_title('Red puncta mask'); axs[2].axis('off')
fig.tight_layout()
fig.savefig('results/segment_red_steps.png', dpi=200)
plt.close(fig)

## Create outlines and overlay on the original image
- Green outlines for red blobs (b)
- Yellow outline for the mixed-channel object (a)
Both a pixel-wise baked overlay and a Matplotlib contour overlay are saved to disk.

In [6]:
# Compute 1-pixel outlines
bd_large = segmentation.find_boundaries(bin_large, mode='outer')
bd_red = segmentation.find_boundaries(bin_red, mode='outer')
imsave('results/boundary_large.png', to_uint8(bd_large))
imsave('results/boundary_red.png', to_uint8(bd_red))

# Baked overlay by painting the outline pixels
overlay = image_rgb.astype(np.float32) / 255.0
overlay[bd_red] = [0.0, 1.0, 0.0]      # green for red-channel blobs
overlay[bd_large] = [1.0, 1.0, 0.0]    # yellow for mixed-channel object
imsave('results/overlay_outlines_baked.png', to_uint8(overlay))

# Matplotlib vector overlay
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(image_rgb)
from skimage.measure import find_contours
for c in find_contours(bin_red.astype(float), 0.5):
    ax.plot(c[:, 1], c[:, 0], color='lime', linewidth=1.5, label='_nolegend_')
for c in find_contours(bin_large.astype(float), 0.5):
    ax.plot(c[:, 1], c[:, 0], color='yellow', linewidth=2.0, label='_nolegend_')
ax.set_axis_off()
fig.tight_layout(pad=0)
fig.savefig('results/overlay_matplotlib.png', dpi=200, bbox_inches='tight', pad_inches=0)
plt.close(fig)

/opt/hostedtoolcache/Python/3.13.7/x64/lib/python3.13/site-packages/skimage/_shared/utils.py:328: UserWarning: results/boundary_large.png is a low contrast image
  return func(*args, **kwargs)
